In [14]:
import pandas as pd
import ast

In [15]:
# Linking to read from csv
colors = pd.read_csv('./datasets/colors.csv')
subject = pd.read_csv('./datasets/subject.csv')
dates = pd.read_fwf('./datasets/dates.txt',
                    header = None)


In [16]:
# Split data, only after first occurrence of '" '
dates[['Title', 'Date']] = dates[0].str.extract(r'"(.*?)" \((.*?)\)')

# Add  separate column for month
dates['Month'] = dates['Date'].str.extract(r'(\w+) \d+, \d+')

# Remove extra column
del dates[0]

In [17]:
# # Remove unnamed column
# del colors['Unnamed: 0']

# Renaming columns to a common name 'Title' for consistency
dates.rename(columns={'Title': 'Title'}, inplace=True)
subject.rename(columns={'TITLE': 'Title'}, inplace=True)
colors.rename(columns={'painting_title': 'Title'}, inplace=True)

# Standardize content of common column to title case
dates['Title'] = dates['Title'].str.title()
subject['Title'] = subject['Title'].str.title()
colors['Title'] = colors['Title'].str.title()

# Remove 'frames' data
subject = subject.loc[:, ~subject.columns.str.contains('FRAME', case=False)]

# Standardize content of all columns in `subject` to title case
subject = subject.rename(columns = str.title)


# Remove quotes from 'Title' for consistency before merging
subject['Title'] = subject['Title'].str.replace('"', '')

# Merging based on the standardized common column 'Title'
merged = pd.merge(dates, colors, on='Title')
merged = pd.merge(merged, subject, on='Title')

# Convert string representation of lists to actual lists
merged['colors'] = merged['colors'].apply(ast.literal_eval)

# Replace the '\r\n' in the lists
merged['colors'] = merged['colors'].apply(lambda x: [color.replace('\r\n', '') for color in x])

# Function to remove square brackets and quotes
def format_colors(colors_str):
    # Remove square brackets and single quotes and split the string into a list
    colors_list = colors_str.strip("[]").replace("'", "").split(', ')
    return ', '.join(colors_list)  # Join list elements into a string

# Convert list back to string, otherwise can't load into MySQL table
merged['colors'] = merged['colors'].apply(str)

# Format string lists
merged['colors'] = merged['colors'].apply(format_colors)
merged['color_hex'] = merged['color_hex'].apply(format_colors)

# Drop the 'season' and 'episode' columns
merged.drop(['season', 'episode'], axis=1, inplace=True)

# Convert the date string to a Pandas datetime object
date = pd.to_datetime(merged['Date'])

# Format the date in MySQL format "YYYY-MM-DD"
merged['Date'] = date.dt.strftime('%Y-%m-%d')


In [18]:
# List of columns representing subjects
subject_columns = ['Aurora_Borealis', 'Barn', 'Beach', 'Boat', 'Bridge',
       'Building', 'Bushes', 'Cabin', 'Cactus', 'Cirrus', 'Cliff', 'Clouds',
       'Conifer', 'Cumulus', 'Deciduous', 'Diane_Andre', 'Dock', 'Farm',
       'Fence', 'Fire', 'Flowers', 'Fog', 'Grass', 'Guest', 'Hills', 'Lake',
       'Lakes', 'Lighthouse', 'Mill', 'Moon', 'Mountain', 'Mountains', 'Night',
       'Ocean', 'Palm_Trees', 'Path', 'Person', 'Portrait', 'River', 'Rocks',
       'Snow', 'Snowy_Mountain', 'Steve_Ross', 'Structure', 'Sun', 'Tree',
       'Trees', 'Waterfall', 'Waves', 'Windmill', 'Winter']

# Create a new 'subject' column containing a comma-separated list of subjects where value is 1
# Not gonna lie I feel like a genius for this one
merged['subject'] = merged.apply(lambda row: ', '.join([column for column in subject_columns if row[column] == 1]), axis=1)

# Reorder columns
new_order = ['Title', 'Date', 'Month', 'Episode', 'id', 'painting_index', 'img_src',
       'youtube_src',  'num_colors', 'colors', 'subject', 'color_hex', 'Black_Gesso', 'Bright_Red',
       'Burnt_Umber', 'Cadmium_Yellow', 'Dark_Sienna', 'Indian_Red',
       'Indian_Yellow', 'Liquid_Black', 'Liquid_Clear', 'Midnight_Black',
       'Phthalo_Blue', 'Phthalo_Green', 'Prussian_Blue', 'Sap_Green',
       'Titanium_White', 'Van_Dyke_Brown', 'Yellow_Ochre', 'Alizarin_Crimson',
       'Aurora_Borealis', 'Barn', 'Beach', 'Boat', 'Bridge',
       'Building', 'Bushes', 'Cabin', 'Cactus', 'Cirrus', 'Cliff', 'Clouds',
       'Conifer', 'Cumulus', 'Deciduous', 'Diane_Andre', 'Dock', 'Farm',
       'Fence', 'Fire', 'Flowers', 'Fog', 'Grass', 'Guest', 'Hills', 'Lake',
       'Lakes', 'Lighthouse', 'Mill', 'Moon', 'Mountain', 'Mountains', 'Night',
       'Ocean', 'Palm_Trees', 'Path', 'Person', 'Portrait', 'River', 'Rocks',
       'Snow', 'Snowy_Mountain', 'Steve_Ross', 'Structure', 'Sun', 'Tree',
       'Trees', 'Waterfall', 'Waves', 'Windmill', 'Winter']
merged = merged[new_order]


# Save to CSV
merged.to_csv('bob_rocks.csv')


In [19]:
merged.columns

Index(['Title', 'Date', 'Month', 'Episode', 'id', 'painting_index', 'img_src',
       'youtube_src', 'num_colors', 'colors', 'subject', 'color_hex',
       'Black_Gesso', 'Bright_Red', 'Burnt_Umber', 'Cadmium_Yellow',
       'Dark_Sienna', 'Indian_Red', 'Indian_Yellow', 'Liquid_Black',
       'Liquid_Clear', 'Midnight_Black', 'Phthalo_Blue', 'Phthalo_Green',
       'Prussian_Blue', 'Sap_Green', 'Titanium_White', 'Van_Dyke_Brown',
       'Yellow_Ochre', 'Alizarin_Crimson', 'Aurora_Borealis', 'Barn', 'Beach',
       'Boat', 'Bridge', 'Building', 'Bushes', 'Cabin', 'Cactus', 'Cirrus',
       'Cliff', 'Clouds', 'Conifer', 'Cumulus', 'Deciduous', 'Diane_Andre',
       'Dock', 'Farm', 'Fence', 'Fire', 'Flowers', 'Fog', 'Grass', 'Guest',
       'Hills', 'Lake', 'Lakes', 'Lighthouse', 'Mill', 'Moon', 'Mountain',
       'Mountains', 'Night', 'Ocean', 'Palm_Trees', 'Path', 'Person',
       'Portrait', 'River', 'Rocks', 'Snow', 'Snowy_Mountain', 'Steve_Ross',
       'Structure', 'Sun', 'Tree', 'T